# 09 추천 시스템
## 04 잠재 요인 협업 필터링

잠재 요인 협업 필터링은 사용자-아이템 평점 매트릭스 속에 숨어있는 잠재 요인을 추출해 추천 예측을 할 수 있게 하는 기법이다. 대규모 다차원 행렬을 SVD와 같은 차원 감소 기법으로 분해하는 과정에서 잠재 요인을 추출하는데, 이러한 기법을 행렬 분해(matrix factorization)라고 한다. 행렬 분해 기반의 잠재 요인 협업 필터링은 넷플릭스 경연 대회에서 사용되며 유명해졌다.

#### 행렬 분해를 통한 잠재 요인 협업 필터링

+ 잠재 요인 협업 필터링의 행렬 분해
    + $R \simeq P Q^T$ = (사용자-잠재요인 행렬)*(잠재요인 - 아이템 행렬)'
+ 잠재 요인 협업 필터링의 행렬 분해 목표는 희소 행렬 형태의 사용자-아이템 평점 행렬을 밀집(Dense) 행렬 형태의 사용자-잠재 요인 행렬과 잠재 요인-아이템 행렬로 분해한 뒤 이를 재 결합하여 밀집 행렬 형태의 사용자-아이템 평점 행렬을 생성하여 사용자에게 새로운 아이템을 추천하는 것이다.

행렬 분해에 의해 추출되는 '잠재 요인'이 정확히 어떤 것인지는 알 수 없지만, 가령 영화 평점 기반의 사용자 - 아이템 기반 데이터라면 영화가 가지는 장르별 특성 선호도로 가정할 수 있다. 즉, 사용자-잠재 요인 행렬은 사용자의 영화 장르에 대한 선호도로, 아이템 - 잠재 요인 행렬은 영화의 장르별 특성값으로 정의할 수 있다.

![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FSpVJY%2FbtroJBbrU6i%2FFXkb4hTnacl4yjkPyEHuC1%2Fimg.png)

사용자 - 잠재요인 행렬에서 FACTOR 1 은 액션 영화 선호도, FACTOR 2 는 로맨스 영화 선호도 이다.  P(u,k)에서 u는 사용자 아이디 k는 잠재요인 칼럼인 장르별 선호도로 정의한다. P(1,1) 는  0.94 P(1,2) 0.96 이다. Q 행렬을 위 P값과 내적 계산을 통해 예측 평점을 계산하기 위해 Q의 행과 열 위치를 서로 교환한 Q.T로 변환한다. 즉, Q(i,k) 는 Q.T(k,i) 이다. 

평점이란 사용자의 특정 영화 장르에 대한 선호도와 개별 영화의 그 자르적 특성값을 반영해 결정된다고 생각할 수 있다. 예를 들어 사용자가 액션 영화를 매우 좋아하고, 특정 영화가 액션 영화의 특성이 매우 크다면 사용자가 해당 영화에 높은 평점을 줄것이다.따라서 평점은 사용자의 장르별 선호도 벡터와 영화의 장르별 특성 벡터를 서로 곱해서 만들 수 있다. 즉, user 1의 item 1의 평점인 R(1, 1)의 4점은 P 매트릭스의 User 1벡터와 Q.T 매트릭스의 Item 2벡터의 내적 결과 값인 2.56ㅇ르로 예측할 수 있다.

#### 요약
+ Factor의 개수 K는 분석가가 정해주어야함
+ P의 개별 행은 개별 사용자에 대한 잠재요인을 반영
+ Q의 개별 열은 개별 아이템에 대한 잠재요인을 반영
+ 아직 사용자가 평점을 매개지 않은 아이템 [i, j]에 대한 예측평점 = (P의 i 행 벡터)*(Q의 j열 벡터)'
![](https://velog.velcdn.com/images%2Fwow-kim%2Fpost%2F09dd4adb-abb1-41bd-853a-7d912b776969%2FSVD.PNG)

#### 사용자-아이템 평점 행렬 분해 이슈

그러나 SVD는 Missing Value가 없는 행렬에 적용 가능하다. 따라서 P와 Q 행렬을 일반적인 SVD 방식으로는 분해할 수 없다.

경사 하강법(Gradient Descent)를 이용하여 P와 Q에 기반한 예측 R값이 실제 R값과 가장 최소의 오류를 가질 수 있도록 비용함수 최적화를 통해 P와 Q를 최적화 유추한다.

#### 확률적 경사 하강법을 이용한 행렬 분해

경사 하강법을 이용한 행렬 분해 방법은 P와 Q 행렬로 계산된 예측 R 행렬 값이 실제 R 행렬 값과 가장 최소의 오류를 가질 수 있도록 반복적인 비용 함수 최적화를 통해 P와 Q를 유추해내는 것이다.

+ 경사 하강법 기반의 행렬 분해 순서
    1. P와 Q를 임의의 값을 가진 행렬로 설정한다.
    2. P와 Q.T 값을 곱해 예측 R 행렬을 계산하고 예측 R 행렬과 실제 R 행렬에 해당하는 오류 값을 계산한다.
    3. 이 오류 값을 최소화할 수 있도록 P와 Q 행렬을 적절한 값으로 각각 업데이트한다.
    4. 만족할 만한 오류 값을 가질 때 까지 2,3번 작업을 반복하면서 P와 Q값을 업데이트해 근사화한다.

#### 경사 하강법 기반의 행렬 분해 비용 함수

+ 실제 값과 예측값의 오류 최소화와 L2 규제(Regularization)를 고려한 비용 함수식
  + $min \sum (\gamma_{u, i} - p_u q_i^t)^2 + \lambda(||q_i||^2 + ||p_u||^2)$
  +  실제값과 예측값의 오류 최소화 + 과적합 개선을 위한 L2 규제

p'와 q'를 업데이트 하며 위의 비용 함수를 최소화한다.

#### 파이썬 코드를 이용한 경사하강법 기반의 행렬 분해 구현과 협업 필터링 기반 영화 추천 구현 실습

이제 SGD를 이용해 행렬 분해를 수행하는 예제를 파이썬으로 구현해본다. 분해하려는 원본 행렬 R을 P와 Q로 분해한 뒤에 다시 P와 Q.T의 내적으로 예측 행렬을 만드는 예제이다.

In [ ]:
import numpy as np

# 원본 행렬 R 생성, 분해 행렬 P와 Q 초기화, 잠재요인 차원 K는 3 설정. 
R = np.array([[4, np.NaN, np.NaN, 2, np.NaN ],
              [np.NaN, 5, np.NaN, 3, 1 ],
              [np.NaN, np.NaN, 3, 4, 4 ],
              [5, 2, 1, 2, np.NaN ]])
num_users, num_items = R.shape
K=3

# P와 Q 매트릭스의 크기를 지정하고 정규분포를 가진 random한 값으로 입력합니다. 
np.random.seed(1)
P = np.random.normal(scale=1./K, size=(num_users, K))
Q = np.random.normal(scale=1./K, size=(num_items, K))

이제 실제 R 행렬과 예측 행렬의 오차를 구하는 get_rmse() 함수를 만든다. get_rmse() 함수는 실제 R 행렬의 Null이 아닌 행렬 값의 위치 인덱스를 추출해 이 인덱스에 있는 실제 R 행렬 값과 분해된 P,Q를 이용해 다시 조합된 예측 행렬 값의 RMSE 값을 반환한다.

In [ ]:
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두개의 분해된 행렬 P와 Q.T의 내적으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)
    
    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
      
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

이제 SGD 기반으로 행렬 분해를 수행한다. 먼저 R에서 널 값을 제외한 데이터의 행렬 인덱스를 추출한다. setps는 SGD를 반복해서 업데이트할 횟수를 의미하며, learning_rate는 SGD의 학습률, r_lambda는 L2 Regularization 계수이다. steps=1000번 동안 반복하면서 새로운 p,q값으로 업데이트 한다. 그리고 get_rmse() 함수를 통해 50회 반복할 때마다 오류 값을 출력한다.

In [ ]:
# R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장. 
non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]

steps=1000
learning_rate=0.01
r_lambda=0.01

# SGD 기법으로 P와 Q 매트릭스를 계속 업데이트. 
for step in range(steps):
    for i, j, r in non_zeros:
        # 실제 값과 예측 값의 차이인 오류 값 구함
        eij = r - np.dot(P[i, :], Q[j, :].T)
        # Regularization을 반영한 SGD 업데이트 공식 적용
        P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
        Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])

    rmse = get_rmse(R, P, Q, non_zeros)
    if (step % 50) == 0 :
        print("### iteration step : ", step," rmse : ", rmse)

### iteration step :  0  rmse :  3.2388050277987723
### iteration step :  50  rmse :  0.4876723101369648
### iteration step :  100  rmse :  0.1564340384819247
### iteration step :  150  rmse :  0.07455141311978046
### iteration step :  200  rmse :  0.04325226798579314
### iteration step :  250  rmse :  0.029248328780878973
### iteration step :  300  rmse :  0.022621116143829466
### iteration step :  350  rmse :  0.019493636196525135
### iteration step :  400  rmse :  0.018022719092132704
### iteration step :  450  rmse :  0.01731968595344266
### iteration step :  500  rmse :  0.016973657887570753
### iteration step :  550  rmse :  0.016796804595895633
### iteration step :  600  rmse :  0.01670132290188466
### iteration step :  650  rmse :  0.01664473691247669
### iteration step :  700  rmse :  0.016605910068210026
### iteration step :  750  rmse :  0.016574200475705
### iteration step :  800  rmse :  0.01654431582921597
### iteration step :  850  rmse :  0.01651375177473524
### iterati

분해된 P와 Q 함수를 P * Q.T 로 예측 행렬을 만들어서 출력해본다.

In [ ]:
pred_matrix = np.dot(P, Q.T)
print('예측 행렬:\n', np.round(pred_matrix, 3))

예측 행렬:
 [[3.991 0.897 1.306 2.002 1.663]
 [6.696 4.978 0.979 2.981 1.003]
 [6.677 0.391 2.987 3.977 3.986]
 [4.968 2.005 1.006 2.017 1.14 ]]


원본 행렬과 비교해 널이 아닌 값은 큰 차이가 나지 않으며, 널인 값은 새로운 예측값으로 채워졌다.

## 행렬 분해를 이용한 잠재 요인 협업 필터링 실습

사용자-아이템 평점 행렬에는사용자가 평점을 매기지 않은 널 데이터가 많기 때문에 주로 SGD나 ALS 기반의 행렬 분해를 이용한다고 말했다. 여기서는 SGD 기반의 행렬 분해를 구현하고 이를 기반으로 사용자에게 영화를 추천해본다.

확률적 경사 하강법을 이용한 행렬 분해 예제의 get_rmse() 함수를 그대로 사용하고 행렬 분해 로직을 새로운 matrix_factorization() 함수로 정리한다.

matrix_factorization(R, K, steps=200, learing_Rate=0.01, r_lambda = 0.01)

+ R : 원본 사용자-아이템 평점 행렬
+ K : 잠재 요인의 차원 수
+ steps : SGD의 반복 횟수
+ learning_Rate는 학습률
+ r_lambda : L2 규제 계수

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두개의 분해된 행렬 P와 Q.T의 내적 곱으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)
    
    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
      
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

In [ ]:
def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda = 0.01):
    num_users, num_items = R.shape
    # P와 Q 매트릭스의 크기를 지정하고 정규분포를 가진 랜덤한 값으로 입력합니다. 
    np.random.seed(1)
    P = np.random.normal(scale=1./K, size=(num_users, K))
    Q = np.random.normal(scale=1./K, size=(num_items, K))

    break_count = 0
       
    # R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트 객체에 저장. 
    non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]
   
    # SGD기법으로 P와 Q 매트릭스를 계속 업데이트. 
    for step in range(steps):
        for i, j, r in non_zeros:
            # 실제 값과 예측 값의 차이인 오류 값 구함
            eij = r - np.dot(P[i, :], Q[j, :].T)
            # Regularization을 반영한 SGD 업데이트 공식 적용
            P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
            Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])
       
        rmse = get_rmse(R, P, Q, non_zeros)
        if (step % 10) == 0 :
            print("### iteration step : ", step," rmse : ", rmse)
            
    return P, Q

영화 평점 행렬 데이터를 새롭게 DataFrame으로 로딩한 뒤에 다시 사용자-아이템 평점 행렬로 만든다.

In [ ]:
import pandas as pd
import numpy as np

movies = pd.read_csv('./ml-latest-small/movies.csv')
ratings = pd.read_csv('./ml-latest-small/ratings.csv')
ratings = ratings[['userId', 'movieId', 'rating']]
ratings_matrix = ratings.pivot_table('rating', index='userId', columns='movieId')

# title 컬럼을 얻기 이해 movies 와 조인 수행
rating_movies = pd.merge(ratings, movies, on='movieId')

# columns='title' 로 title 컬럼으로 pivot 수행. 
ratings_matrix = rating_movies.pivot_table('rating', index='userId', columns='title')

다시 만들어진 사용자-아이템 평점 행렬을 matrix_factorization() 함수를 이용해 행렬 분해한다. 수행 시간이 오래 걸리므로 SGD 반복 횟수인 steps는 200회만 지정한다. 잠재 요인 차원 k는 50, 학습률과 L2 Regularization 계수는 모두 0.01로 설정하고 수행한다.

In [ ]:
P, Q = matrix_factorization(ratings_matrix.values, K=50, steps=200, learning_rate=0.01, r_lambda = 0.01)
pred_matrix = np.dot(P, Q.T)

더 쉽게 영화 아이템 칼럼을 이해하기 위해 반환된 예측 사용자-아이템 평점 행렬을 영화 타이틀을 칼럼명으로 가지는 DataFrame으로 변경한다.

In [ ]:
ratings_pred_matrix = pd.DataFrame(data=pred_matrix, index= ratings_matrix.index,
                                   columns = ratings_matrix.columns)

ratings_pred_matrix.head(3)

이제 만들어진 예측 사용자 - 아이템 평점 행렬 정보를 이용해 개인화된 영화 추천을 해본다. 아이템 기반 최근접 이웃 협업 필터링 실습과 동일한 사용자 아이디 9번에 대한 영화 추천을 이번에는 잠재 요인 협업 필터링으로 추천한다. 아이템 기반 최근접 이웃 협업 필터링 실습에서 사용한 get_unseen_movies() 함수와 recomm_movie_by_userid() 함수를 다시 이용해 추천 영화를 추출한다.

In [ ]:
def get_unseen_movies(ratings_matrix, userId):
    # userId로 입력받은 사용자의 모든 영화정보 추출하여 Series로 반환함. 
    # 반환된 user_rating 은 영화명(title)을 index로 가지는 Series 객체임. 
    user_rating = ratings_matrix.loc[userId,:]
    
    # user_rating이 0보다 크면 기존에 관람한 영화임. 대상 index를 추출하여 list 객체로 만듬
    already_seen = user_rating[ user_rating > 0].index.tolist()
    
    # 모든 영화명을 list 객체로 만듬. 
    movies_list = ratings_matrix.columns.tolist()
    
    # list comprehension으로 already_seen에 해당하는 movie는 movies_list에서 제외함. 
    unseen_list = [ movie for movie in movies_list if movie not in already_seen]
    
    return unseen_list

In [ ]:
def recomm_movie_by_userid(pred_df, userId, unseen_list, top_n=10):
    # 예측 평점 DataFrame에서 사용자id index와 unseen_list로 들어온 영화명 컬럼을 추출하여
    # 가장 예측 평점이 높은 순으로 정렬함. 
    recomm_movies = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_movies

In [ ]:
# 사용자가 관람하지 않는 영화명 추출   
unseen_list = get_unseen_movies(ratings_matrix, 9)

# 아이템 기반의 인접 이웃 협업 필터링으로 영화 추천 
recomm_movies = recomm_movie_by_userid(ratings_pred_matrix, 9, unseen_list, top_n=10)

# 평점 데이타를 DataFrame으로 생성. 
recomm_movies = pd.DataFrame(data=recomm_movies.values,index=recomm_movies.index,columns=['pred_score'])
recomm_movies

아이템 기반 협업 필터링 결과와는 추천된 영화가 많이 다르다. 약간 어둡고 무거운 주제의 영화가 추천되었다.